In [3]:
import os
import sys
import csv
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

repo_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(repo_root)


In [4]:
EXPERIMENT_PATH = os.path.join(repo_root, 'exp', 'ex04')
os.chdir(EXPERIMENT_PATH)

In [5]:
def file_handlers(): 
    data_path = os.path.join(os.getcwd(), 'data')

    all_results = []
    
    for file in os.listdir(data_path):
        id, _ = file.split('_data.csv')
        if file.endswith('.csv'):
      
            file_path = os.path.join(data_path, file)
            df = pd.read_csv(file_path)

            result = {
                    'dataset':id, 
                    'Ours': df['Ours'].mean(), 
                    'Zermello': df['Zermello'].mean(),
                    'higher_order_speedup': df['Zermello'].mean() / df['Ours'].mean(),
                    'Ours_BIN': df['Ours_bin'].mean(),
                    'Zermello_BIN' : df['Zermello_bin'].mean(),
                    'binary_speedup': df['Zermello_bin'].mean() / df['Ours_bin'].mean()
                      }

            all_results.append(result)

    final_df = pd.DataFrame(all_results)
    final_df['dataset'] = pd.to_numeric(final_df['dataset'], errors='coerce')
    return final_df.sort_values(by='dataset')

In [6]:
df = file_handlers()
df.to_csv('results/Convergence_Table.csv')
display(df.style.hide(axis='index'))

dataset,Ours,Zermello,higher_order_speedup,Ours_BIN,Zermello_BIN,binary_speedup
1,9.540000,50.640000,5.308176,9.060000,87.380000,9.644592
2,11.880000,59.460000,5.005051,11.000000,124.060000,11.278182
3,7.280000,13.280000,1.824176,7.160000,51.100000,7.136872
4,6.940000,20.960000,3.020173,7.040000,77.360000,10.988636
5,6.940000,533.240000,76.835735,7.080000,2309.640000,326.220339
6,7.560000,17.560000,2.322751,7.300000,36.620000,5.016438
8,8.280000,43.580000,5.263285,8.220000,111.580000,13.574209
9,19.000000,2769.660000,145.771579,19.000000,2769.660000,145.771579
10,9.440000,21.900000,2.319915,9.260000,155.200000,16.760259
